In [262]:
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##### scraping gold price from 1992 - 2017 (bs4)

In [9]:
header = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
gold_base_url = "https://www.usagold.com/daily-gold-price-history/?ddYears="
years = list(range(1992,2018,1))
gold_date = []
gold_price = []
gold_url = []

# defining multiple urls for scraping gold price from each year (1992 - 2017)
for year in years:
    gold_url.append(gold_base_url+str(year))

# start the scraping for each generated url
for url in gold_url:
    r = requests.get(url, headers = header)
    c = r.content
    soup = bs4(c, "html.parser")
    prices = soup.find("div",{"id":"quotes"})
    table = prices.find("table", {"id":"pricehistorytable"})
    all = table.find_all("tr")[2:]
    for i in all:
        try:
            date = i.find_all("td",{"class":"text"})[0].text
        except:
            date = None
        try:
            price = i.find_all("td",{"class":"text"})[1].text
        except:
            price = None
        gold_date.append(date)
        gold_price.append(price)

gold = pd.DataFrame()
gold["date"] = gold_date
gold["price"] = gold_price        
gold.to_csv("gold price.csv", index = False)

In [10]:
display(gold.shape)
display(gold.isna().sum())
display(gold.head(2))
display(gold.tail(2))

(6715, 2)

date     0
price    0
dtype: int64

,date,price
0,31 Dec 1992,333.00
1,30 Dec 1992,333.15


,date,price
6713,02 Jan 2017,1150.27
6714,30 Dec 2016,1152.07


##### scraping silver price from 1992 - 2017 (bs4)

In [11]:
header = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
silver_base_url = "https://www.usagold.com/daily-silver-price-history/?ddYears="
years = list(range(1992,2018,1))
silver_date = []
silver_price = []
silver_url = []

# defining multiple urls for scraping gold price from each year (1992 - 2017)
for year in years:
    silver_url.append(silver_base_url+str(year))

# start the scraping for each generated url
for url in silver_url:
    r = requests.get(url, headers = header)
    c = r.content
    soup = bs4(c, "html.parser")
    prices = soup.find("div",{"id":"quotes"})
    table = prices.find("table", {"id":"pricehistorytable"})
    all = table.find_all("tr")[2:]
    for i in all:
        try:
            date = i.find_all("td",{"class":"text"})[0].text
        except:
            date = None
        try:
            price = i.find_all("td",{"class":"text"})[1].text
        except:
            price = None
        silver_date.append(date)
        silver_price.append(price)

silver = pd.DataFrame()
silver["date"] = silver_date
silver["price"] = silver_price
silver.to_csv("silver price.csv", index = False)

In [12]:
display(silver.shape)
display(silver.isna().sum())
display(silver.head(2))
display(silver.tail(2))

(6676, 2)

date     0
price    0
dtype: int64

,date,price
0,31 Dec 1992,3.68
1,30 Dec 1992,3.68


,date,price
6674,02 Jan 2017,15.97
6675,30 Dec 2016,16.47


##### scraping stock prices from yahoo finance (bs4 + selenium)

In [95]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
# importing time to make delays
import time

using selenium, because some clicks, user inputs, and scrolling motions in the webpage are necessary in order to display the full data of the table. It's a dynamic page where you need to scroll down through the page in order to display all the data. If we don't scroll it, bs4 won't get the complete page source

In [149]:
def scraping_yfinance(url):
    # --disabling browser popup notifications--
    options = Options()
    options.add_argument("--disable-notifications")
    # to prevent from being spotted as a robot
    options.add_argument('--disable-gpu')
    options.add_argument('user-agent=fake-useragent')
    # installing chromedriver, so that we dont need to keep the chromedriver file
    # that needs to be updated every once in a while. better install the latest automatically
    driver = webdriver.Chrome(ChromeDriverManager().install())
    # opens the browser, maximize window size
    driver.maximize_window()
    # opening url
    driver.get(url)
    # wait a bit until the "I accept" popup shows
    time.sleep(2)
    accept_xpath = "/html/body/div/div/div/div/form/div[2]/div[2]/button"
    accept = driver.find_element(By.XPATH, accept_xpath)
    # clicking "I accept", then wait a bit
    accept.click()
    time.sleep(1)
    later_xpath = "/html/body/div[1]/div/div/div[1]/div/div[4]/div/div/div[1]/div/div/div/div/div/section/button[2]"
    # clicking another popup
    later = driver.find_element(By.XPATH, later_xpath)
    later.click()
    time.sleep(2)
    # since the page is dynamic, meaning the full table wont be shown unless we scroll it,
    # we use selenium to scroll through the page until the bottom page
    # first, getting the page source before scrolling. we will compare it with the
    # page source after scrolling.
    # if html_before != html_after, it means the source is changed after scrolling.
    # in that case, we want to keep scrolling until html_before == html_after
    html1 = driver.page_source
    end = False
    while not end:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, window.scrollY + 1500)") 
        time.sleep(.2)
        driver.execute_script("window.scrollTo(0, window.scrollY + 1500)") 
        time.sleep(.2)
        driver.execute_script("window.scrollTo(0, window.scrollY + 1500)") 
        time.sleep(.2)
        # getting page source after scrolling
        html2 = driver.page_source
        # comparing html_before and html_after
        if html1 == html2:
            end = True
        else:
            html1 = html2
    # YESSS!
    # now that we have the complete page source, we scrape with beautiful soup
    html = driver.page_source
    soup = bs4(html)
    # locating price table
    table = soup.find("div", {"class":"Pb(10px) Ovx(a) W(100%)"})
    # getting all rows from that table (including dates, open price, closing price, etc)
    all = table.find_all("tr", {"class":"BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"})
    dates = []
    prices = []
    df = pd.DataFrame()
    # use loop to get only the 1st and 4th column of each row (date & close price),
    # then save it in lists
    for i in all:
        try:
            date = i.find_all("td")[0].text
        except:
            date = None
        dates.append(date)
        try:
            price = i.find_all("td")[4].text
        except:
            price = None
        prices.append(price)
    # make a dataframe with existing lists
    df["date"] = dates
    df["price"] = prices
    return df

not sure how to use loop in this case, but for now this'll do. preparing empty dataframes to then be filled/overwritten by dataframes created by the function

In [145]:
dowjones_ind_url = "https://finance.yahoo.com/quote/%5EDJI/history?period1=696902400&period2=1514678400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
dowjones_trans_url = "https://finance.yahoo.com/quote/%5EDJT/history?period1=696902400&period2=1514678400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
nasdaq_url = "https://finance.yahoo.com/quote/%5ENDX/history?period1=694224000&period2=1514678400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
snp_url = "https://finance.yahoo.com/quote/%5EGSPC/history?period1=694224000&period2=1514678400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

dj_ind = pd.DataFrame()
dj_trans = pd.DataFrame()
nasdaq = pd.DataFrame()
snp = pd.DataFrame()

In [146]:
dj_ind = scraping_yfinance(dowjones_ind_url)
dj_trans = scraping_yfinance(dowjones_trans_url)
nasdaq = scraping_yfinance(nasdaq_url)
snp = scraping_yfinance(snp_url)

C:\Users\mdima\AppData\Local\Temp\ipykernel_6784\2814194222.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\mdima\AppData\Local\Temp\ipykernel_6784\2814194222.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\mdima\AppData\Local\Temp\ipykernel_6784\2814194222.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\mdima\AppData\Local\Temp\ipykernel_6784\2814194222.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [148]:
display(dj_ind.head(1))
display(dj_trans.head(1))
display(nasdaq.head(1))
display(snp.head(1))

,date,price
0,"Dec 29, 2017","24,719.22"


,date,price
0,"Dec 29, 2017","10,612.29"


,date,price
0,"Dec 29, 2017","6,396.42"


,date,price
0,"Dec 29, 2017","2,673.61"


saving the files in csv so you don't need to rerun the selenium again, since it takes a bit of time to let the computer run automatically

In [150]:
dj_ind.to_csv("dow jones industrial stock price.csv", index = False)
dj_trans.to_csv("dow jones tansportation stock price.csv", index = False)
nasdaq.to_csv("nasdaq stock price.csv", index = False)
snp.to_csv("snp stock price.csv", index = False)

All other datas are collected through downloading csv files, so there's no need to do any scraping there. <br>
Next step will be explained in more detail in "STEP 2 - create sql database"